In [ ]:
import torch
import pandas as pd

from tqdm import tqdm
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM

### Tokenizer and Model

In [2]:
model_id = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

In [93]:
model.config

LlamaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pad_token_id": 128004,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": [
      "lm_head",
      "multi_modal_p

### Data

In [94]:
DATA_DIR = Path.cwd() / "am_reasoning" / "data_files"

In [95]:
df = pd.read_csv((DATA_DIR / "pe_dataset.csv"), index_col=0)

In [111]:
df.columns

Index(['split', 'title', 'argument_bound_1', 'argument_bound_2', 'argument_id',
       'ac', 'label', 'essay'],
      dtype='object')

In [210]:
df.shape

(6089, 8)

### Template Messages

In [ ]:
# sys_msg = """You are an expert in argumentation analysis. You will be given a complete argument. Your task is to analyze the argument holistically and classify it as exactly ONE of the following categories: Position, Claim, or Premise. 

# First, think through your analysis step by step, considering the function and structure of the argument. Explain your reasoning, identifying key characteristics that point to a specific classification.

# Then, provide your final classification as a single word: Position, Claim, or Premise."""

In [ ]:
# instruction = """### You are an expert in Argument Mining. You are given an argument component from an essay. Your task is to classify the argument component as either "MajorClaim", "Claim" or "Premise". You must return the argument component type in following JSON format:: {"component_type": [component_type (str)]}


# """

In [267]:
reasons_d = {}

In [268]:
instruction = """### You are an expert in Argument Mining and discourse analysis. 

INPUT:
- You are given an argument component from an essay. 
- You are given the full text of the essay.

TASK:
- Your task is to classify the whole argument component, as ONLY ONE of: "MajorClaim", "Claim" or "Premise".
- Explain your reasoning step-by-step before making your final classification.

Then, return your final classification in the following JSON format:
{"component_type": [Answer]}

Example:

{"component_type": ["Claim"]}

"""

In [269]:
df_s = df.sample(10)

In [270]:
sys_msgs = []
user_msgs = []

for _, row in df_s.iterrows():

    sys_msg = {"role": "system", "content": instruction}
    usr_msg = {"role": "user", "content": f"""### Here is the argument component to be classified: {row['ac']}"""} 
    
    
    sys_msgs.append(sys_msg)
    user_msgs.append(usr_msg)
    
    #usr_msg = f"""### Here is the complete essay text: {df.iloc[0]['essay']}""" + "\n\n" + f"""### Here is the argument component to be classified: {df.iloc[0]['ac']}"""

In [271]:
messages = []

for i in range(len(sys_msgs)):
    messages.append([sys_msgs[i], user_msgs[i]])

In [272]:
len(messages)

10

In [273]:
messages[5][1]['content']

"### Here is the argument component to be classified: Those who don't even take a word of criticism will eventually lose chances to get improved"

In [274]:
#print(usr_msg)

In [275]:
# messages = [
#     {"role": "system", "content": instruction},
#     {"role": "user", "content": usr_msg},
# ]

In [277]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [278]:
for message in tqdm(messages):

    input_ids = tokenizer.apply_chat_template(message, add_generation_prompt=True, return_tensors="pt").to(model.device) # type: ignore
    outputs = model.generate(input_ids, max_new_tokens=1024, eos_token_id=terminators, do_sample=True, temperature=0.1, top_p=0.9)
    
    response = outputs[0][input_ids.shape[-1]:]
    reasons_d[message[1]['content']] = tokenizer.decode(response, skip_special_tokens=True)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [01:24<00:00,  8.44s/it]


In [279]:
reasons_d

{'### Here is the argument component to be classified: we have to be aware to never judge with their appearance': '### Classification of the Argument Component\n\nTo classify the argument component, let\'s break it down step by step:\n\n1. **Understanding the Argument Component**: The argument component is a statement that presents a claim or a position. In this case, it\'s "we have to be aware to never judge with their appearance".\n\n2. **Identifying the Premise**: The first step in argument mining is to identify the premise, which is the foundation of the argument. In this case, the premise is "we have to be aware".\n\n3. **Analyzing the Premise**: The premise "we have to be aware" can be further broken down into two parts:\n   - **Subject**: "we" (referring to the speaker or the group)\n   - **Object**: "be aware" (referring to the action or the state)\n\n4. **Identifying the Premise\'s Role**: The phrase "we have to be aware" implies that there is a responsibility or a duty to be 

In [280]:
reasons_l = [v for k,v in reasons_d.items()]

In [281]:
reasons_l

['### Classification of the Argument Component\n\nTo classify the argument component, let\'s break it down step by step:\n\n1. **Understanding the Argument Component**: The argument component is a statement that presents a claim or a position. In this case, it\'s "we have to be aware to never judge with their appearance".\n\n2. **Identifying the Premise**: The first step in argument mining is to identify the premise, which is the foundation of the argument. In this case, the premise is "we have to be aware".\n\n3. **Analyzing the Premise**: The premise "we have to be aware" can be further broken down into two parts:\n   - **Subject**: "we" (referring to the speaker or the group)\n   - **Object**: "be aware" (referring to the action or the state)\n\n4. **Identifying the Premise\'s Role**: The phrase "we have to be aware" implies that there is a responsibility or a duty to be aware. This suggests that the argument is not just about being aware, but also about the responsibility or duty t

In [258]:
# input_ids = tokenizer.apply_chat_template(
#     messages,
#     add_generation_prompt=True,
#     return_tensors="pt"
# ).to(model.device) # type: ignore

In [259]:
#input_ids

In [260]:
#print(tokenizer.decode(input_ids[0]))

In [261]:
# terminators = [
#     tokenizer.eos_token_id,
#     tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

In [262]:
# outputs = model.generate(
#     input_ids,
#     max_new_tokens=1024,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.1,
#     top_p=0.9,
# )

In [263]:
#response = outputs[0][input_ids.shape[-1]:]

In [264]:
#print(tokenizer.decode(response, skip_special_tokens=True))

In [265]:
#reasons_d[usr_msg] = tokenizer.decode(response, skip_special_tokens=True)

In [266]:
#reasons_d